In [4]:
import numpy as np
import pandas as pd
import copy

In [5]:
propensity = np.array([
    [1, 0.2, 0],
    [0.7, 0.3, 0],
    [0.5, 0.5, 0.5],
    [0, 0, 0.3]
])

user_cap = 1
rpc = np.array([[200, 300, 400]])
kpi_targets = np.array([[400, 300, 1200]])

num_product = propensity.shape[-1]

In [6]:
def generate_ads2cust_encoding(shape=(10, 4), num_product=3):
    """
    Encoded Representation in vector format
    Args:
    shape (tuple): Encoded Representation in vector format 
    num_product (int): Number of Products
    """
    # rows represent num cust
    (num_matrices, rows) = shape
    one_positions = np.random.choice(num_product, size=(num_matrices, rows))
    return one_positions

# 3 Ads (0,1,2), 4 People
# [0,0,1,2] -> All users must receive ads 
# User cap must be 1
def generate_ads2cust_encoding2matrix(shape=(10, 4, 3), num_product=3):
    ads2cust_vec = generate_ads2cust_encoding(shape=shape[0:2], num_product=num_product)
    # print(ads2cust_vec)
    (num_matrices, rows, cols) = shape
    ads2cust_matrices = np.zeros((num_matrices, rows, cols), dtype=int)

    batch_idx = np.arange(num_matrices)[:, None]         # shape (10, 1)
    row_idx = np.arange(rows)[None, :]                   # shape (1, 7)

    ads2cust_matrices[batch_idx, row_idx, ads2cust_vec] = 1
    return ads2cust_matrices

def evaluation(
        randomized_matrices, 
        prop=propensity, 
        rev=rpc,
        mode="Kpis",
    ):
    assert rpc is not None
    assert mode is not None

    # Make a deep copy of prop to avoid side effects
    prop_copy = copy.deepcopy(prop)

    # Expand propen_matrix and revenue_vec to broadcast
    actual_propen = prop_copy[None, :, :]      # shape (1, 7, 3)
    rev = rpc[None, :, :]                      # shape (1, 1, 3)
    
    if mode == "Kpis":
        prop_copy[prop_copy != 0] = 1
        highest_revenue_matrix = randomized_matrices * prop_copy * rev
        return highest_revenue_matrix
    if mode == "Revenue":
        expected_revenue_matrix = randomized_matrices * actual_propen * rev
        return expected_revenue_matrix

def eval_fitness_batch_vectorized(
        randomized_matrices, 
        rpc = None,
        mode = None,
        kpi_rev_A = 400, 
        kpi_rev_B = 300, 
        kpi_rev_C = 1200, 
    ):
    if mode == "Kpis":
        sum_highest_matrices = evaluation(randomized_matrices, rpc=None, mode = "Kpis")
        vec_sum = sum_highest_matrices.sum(axis=1)   # (N, 6)
        # Compute num of kpis reached
        viol_A = (vec_sum[:, 0] >= kpi_rev_A).astype(int)
        viol_B = (vec_sum[:, 1] >= kpi_rev_B).astype(int)
        viol_C = (vec_sum[:, 2] >= kpi_rev_C).astype(int)

        # num_reached_kpis
        num_reached_kpis = ((viol_A) + (viol_B) + (viol_C))
        return num_reached_kpis

    if mode == "Revenue":
        sum_expected_matrices = evaluation(randomized_matrices, mode = "Revenue")
        vec_sum = sum_expected_matrices.sum(axis=1)  # (N, 6)

        # Compute raw fitness
        expected_revenue = (vec_sum[:, 0] ) + (vec_sum[:, 1] ) + (vec_sum[:, 2] ) 
        return expected_revenue
    
    else: 
        # Optimize both Kpis and Revenue
        sum_highest_matrices = evaluation(randomized_matrices, mode = "Kpis")
        vec_sum1 = sum_highest_matrices.sum(axis=1)   # (N, 6)
        # Compute num of kpis reached
        viol_A = (vec_sum1[:, 0] >= kpi_rev_A).astype(int)
        viol_B = (vec_sum1[:, 1] >= kpi_rev_B).astype(int)
        viol_C = (vec_sum1[:, 2] >= kpi_rev_C).astype(int)

        # num_reached_kpis
        num_reached_kpis = ((viol_A) + (viol_B) + (viol_C))

        sum_expected_matrices = evaluation(randomized_matrices, mode = "Revenue")
        vec_sum2 = sum_expected_matrices.sum(axis=1)  # (N, 6)

        # Compute revenue fitness
        expected_revenue = (vec_sum2[:, 0] ) + (vec_sum2[:, 1] ) + (vec_sum2[:, 2] ) 

        return expected_revenue + num_reached_kpis * expected_revenue


def breed_solutions(
        randomized_matrices, 
        fitness_vec, 
        mutation_propotion=0.05, 
        top_k=10, 
        upsampling=100
    ):
    # Paired matrix with its fitness value
    matched = list(zip(fitness_vec, randomized_matrices))
    # Rank matrices with its fitness value
    sorted_matched = sorted(matched, key=lambda x: x[0], reverse=True)
    # Select top K solution based on its fitness value from highest to lowest 
    # i.e. select top strongest members in the population
    top_k_matrice = np.array([m for _, m in sorted_matched[:top_k]])  # shape (10, 7, 3)

    # Random Pairs combinations based on upsampling 
    # The upsampling shounding exceed the num of population in randomized_matrices
    pair_indices = np.random.randint(0, top_k, size=(upsampling, 2))
    parents1 = top_k_matrice[pair_indices[:, 0]]
    parents2 = top_k_matrice[pair_indices[:, 1]]

    # Crossover 
    # Swap each matrix using the last row
    offspring1 = np.concatenate([parents1[:, :-1], parents2[:,-1, None]], axis=1)
    offspring2 = np.concatenate([parents2[:,-1, None], parents1[:, :-1]], axis=1)

    # combine all the offsprings together
    offspring = np.concatenate([offspring1, offspring2], axis=0)  # shape: (2*upsampling, 7, 3)
    
    # ---- Mutation: swap column 0 and 1 ---- #
    num_to_mutate = int(len(offspring) * mutation_propotion)
    if num_to_mutate > 0:
        mutate_indices = np.random.choice(len(offspring), size=num_to_mutate, replace=False)

        # Extract affected matrices
        mutated = offspring[mutate_indices]

        # Swap columns 0 and 1
        mutated_swapped = mutated.copy()
        mutated_swapped[:, :, [0, 1]] = mutated[:, :, [1, 0]]

        # Assign back to offspring
        offspring[mutate_indices] = mutated_swapped
    return offspring

In [ ]:
from tqdm import trange

propensity = np.array([
    [1, 0.2, 0],
    [0.7, 0.3, 0],
    [0.5, 0.5, 0.5],
    [0, 0, 0.3]
])
user_cap = 1
rpc = np.array([[200, 300, 400]])
kpi_targets = np.array([[400, 300, 1200]])

num_cust = propensity.shape[0]
num_product = propensity.shape[-1]

randomized_matrices = generate_ads2cust_encoding2matrix(shape=(100, num_cust, num_product), num_product=num_product)
mutation_propotion=0.005
top_k=10
upsampling=100

for episode_num in trange(5):
    if episode_num == 0:
        # The higher population, the more accurate result
        # The higher upsampling, the more accurate result
        # Top K should vary properly to prevent overfitting
        num_population = 50
        randomized_matrices = generate_ads2cust_encoding2matrix(shape=(num_population, num_cust, num_product), num_product=num_product)
        breed_topk = randomized_matrices
        fitness_vec = eval_fitness_batch_vectorized(randomized_matrices, 
                                                    rpc=rpc, 
                                                    kpi_rev_A = 400, 
                                                    kpi_rev_B = 300, 
                                                    kpi_rev_C = 1200,)
        
    breed_topk = breed_solutions(breed_topk, fitness_vec, top_k=20, upsampling=100)

    # expected_revenue_matrices = expected_revenue(randomized_matrices)
    # Should Include stopping criteria 
    # Select the best solution, because max fitness val for each epoch can be fluctuated
    fitness_vec = eval_fitness_batch_vectorized(breed_topk)

    print("Num episode:", episode_num, "\n"
          "Mean Fitness val:", fitness_vec.mean(),
          "Max Fitness val:", fitness_vec.max()
    )

solution = breed_topk[0] 
print("Best Solution from Genetic Algorithm: \n", solution)

100%|██████████| 5/5 [00:00<00:00, 2011.46it/s]

Num episode: 0 
Mean Fitness val: 951.25 Max Fitness val: 1830.0
Num episode: 1 
Mean Fitness val: 1102.2 Max Fitness val: 1830.0
Num episode: 2 
Mean Fitness val: 1133.2 Max Fitness val: 1830.0
Num episode: 3 
Mean Fitness val: 1116.85 Max Fitness val: 1830.0
Num episode: 4 
Mean Fitness val: 1116.85 Max Fitness val: 1830.0
Best Solution from Genetic Algorithm: 
 [[1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]]


In [ ]:
# propensity = np.array([
#     [1, 0.2, 0],
#     [0.7, 0.3, 0],
#     [0.5, 0.5, 0.5],
#     [0, 0, 0.3]
# ])
# user_cap = 1
# rpc = np.array([[200, 300, 400]])
# kpi_targets = np.array([[400, 300, 1200]])

In [8]:
# Should be run with the above cell directly because of some memory conflicts
propensity = np.array([
    [1, 0.2, 0],
    [0.7, 0.3, 0],
    [0.5, 0.5, 0],
    [0, 0.7, 0]
])

rpc = np.array([[200, 300, 1200]])
kpi_targets = np.array([[400, 300, 1200]])

num_cust = propensity.shape[0]
num_product = propensity.shape[-1]

randomized_matrices = generate_ads2cust_encoding2matrix(shape=(100, num_cust, num_product), num_product=num_product)
fitness_vec = eval_fitness_batch_vectorized(randomized_matrices)
mutation_propotion=0.005
top_k=10
upsampling=100

for episode_num in trange(5):
    if episode_num == 0:
        # The higher population, the more accurate result
        # The higher upsampling, the more accurate result
        # Top K should vary properly to prevent overfitting
        num_population = 50
        randomized_matrices = generate_ads2cust_encoding2matrix(shape=(num_population, num_cust, num_product), num_product=num_product)
        breed_topk = randomized_matrices
        fitness_vec = eval_fitness_batch_vectorized(randomized_matrices, 
                                                    rpc=rpc, 
                                                    kpi_rev_A = 400, 
                                                    kpi_rev_B = 300, 
                                                    kpi_rev_C = 1200,)
        
    breed_topk = breed_solutions(breed_topk, fitness_vec, top_k=20, upsampling=100)

    # expected_revenue_matrices = expected_revenue(randomized_matrices)
    # Include stopping criteria
    # Select the best solution, because max fitness val for each epoch can be fluctuated
    fitness_vec = eval_fitness_batch_vectorized(breed_topk)

    print("Num episode:", episode_num, "\n"
          "Mean Fitness val:", fitness_vec.mean(),
          "Max Fitness val:", fitness_vec.max()
    )

solution = breed_topk[0] 
print("\n")
print("Best Solution from Genetic Algorithm: \n", solution)

100%|██████████| 5/5 [00:00<00:00, 3244.86it/s]

Num episode: 0 
Mean Fitness val: 1992.65 Max Fitness val: 3900.0
Num episode: 1 
Mean Fitness val: 2701.8 Max Fitness val: 3900.0
Num episode: 2 
Mean Fitness val: 2838.6 Max Fitness val: 3900.0
Num episode: 3 
Mean Fitness val: 2844.3 Max Fitness val: 3900.0
Num episode: 4 
Mean Fitness val: 2835.75 Max Fitness val: 3900.0


Best Solution from Genetic Algorithm: 
 [[1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]]
